<a href="https://colab.research.google.com/github/richdeport/Classificador-de-Imagens-com-Rede-Neural/blob/main/IARedeNeural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IDENTIFICADOR DE GATOS, CACHORROS E HUMANOS (VERSÃO CORRIGIDA)

# IMPORTAÇÕES
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
from google.colab import files
import matplotlib.pyplot as plt
import cv2

# CARREGA O MODELO PRÉ-TREINADO (ImageNet)
print("🔄 Carregando modelo pré-treinado (MobileNetV2)...")
modelo = tf.keras.applications.MobileNetV2(weights="imagenet")
print("✅ Modelo carregado com sucesso!\n")

# CARREGA O CLASSIFICADOR FACIAL (OpenCV)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# --- FUNÇÃO DE PREDIÇÃO (MODIFICADA E SIMPLIFICADA) ---
def prever_imagem(caminho):
    # --- Exibe a imagem enviada ---
    img_color = cv2.imread(caminho)
    img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)
    plt.imshow(img_rgb)
    plt.axis("off")
    plt.title("🖼️ Imagem enviada")
    plt.show()

    # --- 1ª VERIFICAÇÃO: DETECÇÃO DE ROSTO HUMANO ---
    gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
    rostos = face_cascade.detectMultiScale(gray, 1.3, 5)

    if len(rostos) > 0:
        print("✅ Resultado: HUMANO")
        return

    # --- 2ª VERIFICAÇÃO: ANÁLISE PELA REDE NEURAL ---
    img = image.load_img(caminho, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

    previsao = modelo.predict(img_array)
    # Pegamos apenas a melhor previsão do modelo.
    resultado = tf.keras.applications.mobilenet_v2.decode_predictions(previsao, top=1)[0][0]

    # Extrai o nome e a confiança da melhor previsão
    nome_predicao = resultado[1].lower() # ex: 'rhodesian_ridgeback'
    confianca = resultado[2]

    # *** NOVA LÓGICA DE CLASSIFICAÇÃO ***
    # Listas de palavras-chave mais abrangentes, incluindo raças comuns.
    palavras_cachorro = [
        "dog", "retriever", "terrier", "bulldog", "poodle", "beagle", "labrador",
        "spaniel", "husky", "pug", "chihuahua", "shepherd", "hound", "boxer",
        "ridgeback", "rottweiler", "corgi", "shiba" # Adicionamos palavras-chave de raças
    ]
    palavras_gato = [
        "cat", "kitten", "tabby", "siamese", "persian", "egyptian_cat", "lynx"
    ]

    # Limiar de confiança: se o modelo não tiver certeza, não arriscamos.
    LIMIAR_CONFIANCA = 0.15 # 15% de confiança mínima

    if confianca < LIMIAR_CONFIANCA:
        print("❌ Resultado: NÃO IDENTIFICADO")
        return

    # Verificamos se alguma das palavras-chave está no nome da previsão
    eh_cachorro = any(palavra in nome_predicao for palavra in palavras_cachorro)
    eh_gato = any(palavra in nome_predicao for palavra in palavras_gato)

    if eh_cachorro:
        print("✅ Resultado: CACHORRO")
    elif eh_gato:
        print("✅ Resultado: GATO")
    else:
        print("❌ Resultado: NÃO IDENTIFICADO")


# --- BLOCO DE UPLOAD E EXECUÇÃO ---
print("📸 Envie uma imagem (jpg, jpeg, png, etc.) para analisar:\n")
uploaded = files.upload()

for nome_arquivo in uploaded.keys():
    print(f"\n----------------------------------------")
    print(f"Analisando a imagem: {nome_arquivo}")
    prever_imagem(nome_arquivo)
    print(f"----------------------------------------")